In [ ]:
import datetime
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt


from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
!pip install scipy

We define the Alexnet architecture

In [ ]:
@tf.keras.utils.register_keras_serializable()
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= input_shape,
                        kernel_initializer= 'he_normal',kernel_regularizer=regularizers.l2(0.0005)))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal',kernel_regularizer=regularizers.l2(0.0005)))
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal',kernel_regularizer=regularizers.l2(0.0005)))
        self.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal',kernel_regularizer=regularizers.l2(0.0005)))
        self.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal',kernel_regularizer=regularizers.l2(0.0005)))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(num_classes, activation= 'softmax'))

        self.compile(optimizer= tf.keras.optimizers.Adam(base_lr),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    def get_config(self):
        config = super().get_config()
        config.update({
            'input_shape': self.input_shape[1:],
            'num_classes': self.layers[-1].units
        })
        return config


We import the training and test data

In [ ]:
train_dir='imagenet2/train'
val_dir='imagenet2/val'

total_epochs = 60
batch_size = 64
base_lr=0.0003
image_height = 227
image_width = 227

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)


val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(227, 227),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator1 = val_datagen.flow_from_directory(
    val_dir,
    target_size=(227, 227),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


We create an Alexnet model and train it

In [ ]:
model_alex=AlexNet((227,227,3),10)

/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
train_num=train_generator.samples
val_num=val_generator.samples

In [ ]:
def lr_schedule(epoch, lr):
    return base_lr*(1-(epoch/total_epochs))

lr_callback = LearningRateScheduler(lr_schedule)
checkpoint_callback = ModelCheckpoint(
    filepath="alexnet_checkpoint_epoch_{epoch:02d}.h5",
    save_freq='epoch',
    verbose=1
)
tensorboard_callback = TensorBoard(log_dir='./logs')

alex_net = model_alex.fit(
    train_generator,
    epochs=total_epochs,
    steps_per_epoch=train_num // batch_size,
    validation_steps=val_num // batch_size,
    validation_data=val_generator,
    callbacks=[lr_callback, checkpoint_callback, tensorboard_callback]
)

/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2055 - loss: 3.8334

/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: saving model to alexnet_checkpoint_epoch_01.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.2060 - loss: 3.8293 - val_accuracy: 0.3947 - val_loss: 2.7151 - learning_rate: 3.0000e-04
Epoch 2/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 5:08 2s/step - accuracy: 0.4688 - loss: 2.5908

...

147/147 ━━━━━━━━━━━━━━━━━━━━ 42s 271ms/step - accuracy: 1.0000 - loss: 0.3640 - val_accuracy: 0.7795 - val_loss: 1.5238 - learning_rate: 2.5000e-05
Epoch 57/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9874 - loss: 0.3933
Epoch 57: saving model to alexnet_checkpoint_epoch_57.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.9874 - loss: 0.3933 - val_accuracy: 0.7836 - val_loss: 1.5206 - learning_rate: 2.0000e-05
Epoch 58/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 5:39 2s/step - accuracy: 1.0000 - loss: 0.3801
Epoch 58: saving model to alexnet_checkpoint_epoch_58.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 45s 291ms/step - accuracy: 1.0000 - loss: 0.3801 - val_accuracy: 0.7833 - val_loss: 1.5208 - learning_rate: 1.5000e-05
Epoch 59/60
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9917 - loss: 0.3857
Epoch 59: saving model to alexnet_checkpoint_epoch_59.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.9917 - loss: 0.3857 - val_accuracy: 0.7861 - val_loss: 1.5081 - learning_rate: 1.0000e-05
Epoch 60/60
  1/147 ━━━━━━━━━━━━━━━━━━━━ 4:54 2s/step - accuracy: 1.0000 - loss: 0.3784
Epoch 60: saving model to alexnet_checkpoint_epoch_60.h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 42s 270ms/step - accuracy: 1.0000 - loss: 0.3784 - val_accuracy: 0.7859 - val_loss: 1.5107 - learning_rate: 5.0000e-06


In [ ]:
model_alex.save_weights('alex_net_adjusted.weights.h5')


In [ ]:
model_alex.summary()

Model: "alex_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 174,966,944 (667.45 MB)

 Trainable params: 58,322,314 (222.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 116,644,630 (444.96 MB)

In [ ]:
model = AlexNet(input_shape=(227,227,3), num_classes=10)

model.load_weights('alex_net_adjusted.weights.h5')


/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model = AlexNet(input_shape=(227,227,3), num_classes=10)

/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "alex_net_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,322,314 (222.48 MB)

 Trainable params: 58,322,314 (222.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.save_weights('alex_net_small.weights.h5')

In [ ]:
history_alex_dic=alex_net.history

In [ ]:
model = AlexNet(input_shape=(227,227,3), num_classes=10)

model.load_weights('alex_net_new.weights.h5')


In [ ]:
val_steps = val_generator1.samples // batch_size

predictions = model.predict(val_generator1, steps=val_steps)
true_labels = val_generator1.classes[:val_steps * batch_size]

num_classes = len(val_generator1.class_indices)
true_labels_one_hot = tf.keras.utils.to_categorical(true_labels, num_classes)

top1_metric = tf.keras.metrics.CategoricalAccuracy()
top5_metric = tf.keras.metrics.TopKCategoricalAccuracy(k=5)

top1_metric.update_state(true_labels_one_hot, predictions)
top5_metric.update_state(true_labels_one_hot, predictions)

print("Top-1 Accuracy on validation set:", top1_metric.result().numpy())
print("Top-5 Accuracy on validation set:", top5_metric.result().numpy())

/opt/anaconda3/envs/newenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step
Top-1 Accuracy on validation set: 0.7871414
Top-5 Accuracy on validation set: 0.97284836
